In [5]:
import pulp

# Parameters
num_people = 6
num_days = 6
hours_early = 4
hours_late = 4

# Initialize the model
model = pulp.LpProblem("Min_Total_Working_Hours", pulp.LpMinimize)

# Decision variables: x[i][d][s] indicates whether person i works shift s on day d (s=0 for early, s=1 for late)
x = [[[pulp.LpVariable(f"x_{i}_{d}_{s}", cat='Binary') for s in range(2)]
      for d in range(num_days)] for i in range(num_people)]

# Objective function: minimize total working hours
model += pulp.lpSum(
    x[i][d][0] * hours_early + x[i][d][1] * hours_late
    for i in range(num_people)
    for d in range(num_days)
)

# Constraint 1: each person can work at most one shift per day
for i in range(num_people):
    for d in range(num_days):
        model += x[i][d][0] + x[i][d][1] <= 1

# Constraint 2: each shift must have at least one worker per day
for d in range(num_days):
    model += pulp.lpSum(x[i][d][0] for i in range(num_people)) >= 1  # Early shift
    model += pulp.lpSum(x[i][d][1] for i in range(num_people)) >= 1  # Late shift

# Constraint 3: exactly 6 people must work each day
for d in range(num_days):
    model += pulp.lpSum(x[i][d][0] + x[i][d][1] for i in range(num_people)) == 6

# Solve the model
solver = pulp.PULP_CBC_CMD()
result = model.solve(solver)

# Output shift counts per day
if result == pulp.LpStatusOptimal:
    print("Optimal shift schedule found:\n")
    total_hours = 0
    for d in range(num_days):
        early_count = sum(pulp.value(x[i][d][0]) for i in range(num_people))
        late_count  = sum(pulp.value(x[i][d][1]) for i in range(num_people))
        day_hours = early_count * hours_early + late_count * hours_late
        total_hours += day_hours
        print(f"Day {d+1}: Early shift - {int(early_count)} workers, Late shift - {int(late_count)} workers, Total {int(day_hours)} hours")
    print(f"\n Minimum total working hours: {int(total_hours)} hours")
else:
    print("No feasible solution found.")


✅ 找到最小总工时排班方案：

第 1 天: 早班 3 人，晚班 3 人，合计 24 小时
第 2 天: 早班 2 人，晚班 4 人，合计 24 小时
第 3 天: 早班 2 人，晚班 4 人，合计 24 小时
第 4 天: 早班 5 人，晚班 1 人，合计 24 小时
第 5 天: 早班 2 人，晚班 4 人，合计 24 小时
第 6 天: 早班 4 人，晚班 2 人，合计 24 小时

🔢 最小总工作时间: 144 小时
